# 1) Data Exploration Notebook
Key Deliverables - Two datasets (train/test) containing unified views of temperature and energy use across time. 

#### Import Libraries

In [61]:
import pandas as pd
import os
from os.path import dirname

## Load Data Sets

In [62]:
# Point toward data directory
data_directory = dirname(os.getcwd()) + '\\data\\raw'

# Read CSVs
temp_df = pd.read_csv(data_directory + '\\hr_temp_20170201-20200131_subset.csv')
energy_df = pd.read_csv(data_directory + '\\hrl_load_metered - 20170201-20200131.csv')

## View Data Sets

In [63]:
display(temp_df.head())
display(energy_df.head())

,STATION,DATE,REPORT_TYPE,SOURCE,HourlyDryBulbTemperature
0,72520514762,2017-02-01T00:53:00,FM-15,7,37.0
1,72520514762,2017-02-01T01:53:00,FM-15,7,37.0
2,72520514762,2017-02-01T02:53:00,FM-15,7,36.0
3,72520514762,2017-02-01T03:53:00,FM-15,7,36.0
4,72520514762,2017-02-01T04:53:00,FM-15,7,36.0


,datetime_beginning_utc,datetime_beginning_ept,nerc_region,mkt_region,zone,load_area,mw,is_verified
0,2/1/2017 5:00,2/1/2017 0:00,RFC,WEST,DUQ,DUQ,1419.881,True
1,2/1/2017 6:00,2/1/2017 1:00,RFC,WEST,DUQ,DUQ,1379.505,True
2,2/1/2017 7:00,2/1/2017 2:00,RFC,WEST,DUQ,DUQ,1366.106,True
3,2/1/2017 8:00,2/1/2017 3:00,RFC,WEST,DUQ,DUQ,1364.453,True
4,2/1/2017 9:00,2/1/2017 4:00,RFC,WEST,DUQ,DUQ,1391.265,True


## Data Cleaning and Unification

In [64]:
#Subsetting to columns we need
temp_df = temp_df[['DATE','HourlyDryBulbTemperature']].rename(columns = {'HourlyDryBulbTemperature':'temp','DATE':'date'})
energy_df = energy_df[['datetime_beginning_ept','mw']]

#### Converting Date Columns to Datetime Objects
This will allow for better manipulation of these objects and will allow us to extract more granular date information from them. 

In [65]:
#Converting date columns 
temp_df['date'] = pd.to_datetime(temp_df['date'])
energy_df['datetime_beginning_ept'] = pd.to_datetime(energy_df['datetime_beginning_ept'])

#Sorting Values based on date 
temp_df = temp_df.sort_values(by = 'date')
energy_df = energy_df.sort_values(by = 'datetime_beginning_ept')

# Merge data sets on index now that they are sorted by date
master_df = temp_df.merge(energy_df, left_index = True, right_index = True, how = 'inner').drop(columns = ['datetime_beginning_ept'])

#### Data Cleaning
We will interpolate any missing data that is present in the temperature data using linear interpolation. Additionally, we will also extract so more granular datetime information from our date column

In [66]:
# Impute missing temp data
master_df['temp'] = master_df['temp'].interpolate(method = 'linear')

# Creating time based columns
master_df['hour'] = master_df.date.dt.hour
master_df['day'] = master_df.date.dt.day
master_df['weekday'] = master_df.date.dt.weekday
master_df['month'] = master_df.date.dt.month
master_df['year'] = master_df.date.dt.year
master_df['doy'] = master_df.date.dt.dayofyear

#Creating Datetime as index
master_df = master_df.set_index('date')

#### Splitting into Train and Test Data Sets

In [67]:
#Splitting into train and test
test_df = master_df[master_df.index >= '2020-01-01']
train_df = master_df[master_df.index < '2020-01-01']

# First train row is different from that in the manning guide
display(train_df.head(1))
display(test_df.head(1))

,temp,mw,hour,day,weekday,month,year,doy
date,,,,,,,,
2017-02-01 00:53:00,37.0,1419.881,0,1,2,2,2017,32


,temp,mw,hour,day,weekday,month,year,doy
date,,,,,,,,
2020-01-01 00:53:00,31.0,1363.428,0,1,2,1,2020,1


#### Writing Training and Testing Datasets to Interim Directory

In [69]:
# Setting interim directory path
interim_directory = dirname(os.getcwd()) + '\\data\\interim'

# Write Training and Testing Datasets to data//interim
train_df.to_csv(interim_directory + '\\train_data.csv')
test_df.to_csv(interim_directory + '\\test_data.csv')